# Use the Entso-e API to download the data programatically

Download the "week-ahead" data for Germany for: 
- Minimum load forecast per day of week 
- Maximum load forecast per day of week

All units are in MW (Megawatts). 

"Week-ahead" data provides the min and max forecast-values for each day of a given week. This means 14 values per week

In [ ]:
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup as bs

# Import dictionaries, security_token and week_ahead loading script

from Name_convention_dictionaries import DocumentTypeDict, ProcessTypeDict, AreaDict, PsrTypeDict
from security_token import security_token
from Load_data_entsoe_week import Load_data_week_ahead

# Reverse dictionaries for easier understanding

zone_dict = {v:k for k,v in AreaDict.items()}
document_type_dict = {v:k for k,v in DocumentTypeDict.items()}
process_type_dict = {v:k for k,v in ProcessTypeDict.items()}
generation_type_dict = {v:k for k,v in PsrTypeDict.items()}

In [ ]:
# support functions
def str_to_datetime(date_str):
    return datetime.datetime.strptime(date_str, '%Y%m%d%H%M')

def datetime_to_str(date):
    return datetime.datetime.strftime(date, "%Y%m%d%H%M")

## Specify region (country name or bidding zone according to Entso-e) and time horizon
Check Name_convention_dictionaries.py if unsure how the country or bidding zone is named by Entso-e.

In [ ]:
region = 'Germany'

# The Entso-e API only allows to download 380 days per call. So create a loop to download all data 
# Time-format: Year, Month, Day, Hour, Minute

num_of_years_to_download = 7
most_recent_date = '202010252300'
list_for_loop = [365]*num_of_years_to_download

## Download Load Data

In [ ]:
# Define parameters to download the data 
process_type = 'Week ahead'
document_type = 'System total load'

In [ ]:
last_date_to_download = most_recent_date

file_list = []

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # generate url for API call

    url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
          +'&documentType='+document_type_dict[document_type]\
          +'&processType='+process_type_dict[process_type]\
          +'&outBiddingZone_Domain='+zone_dict[region]\
          +'&periodStart='+begin_time\
          +'&periodEnd='+end_time
    
    # load and save data
    
    try:
        file_name = Load_data_week_ahead(url)
        file_list.append(file_name)
        last_date_to_download = begin_time
    except:
        print('\n No more data available')